In [1]:
import pandas as pd
import numpy as np

# Загрузка данных
df1 = pd.read_excel('data/data001.xlsx', skiprows=9)
df2 = pd.read_excel('data/data002.xlsx', skiprows=4)
df3 = pd.read_excel('data/data003.xlsx', skiprows=4)

# --- Обработка первого датасета (df1: пострадавшие) ---
df1 = df1.iloc[:, :45]
df1.columns = [
    'court', 'total_victims', 'physical', 'psychological', 'sexual', 'other',
    'male', 'female', 'boy', 'girl', 'age_0_7', 'age_7_10', 'age_10_14', 'age_14_18',
    'age_18_34', 'age_34_50', 'age_50+', 'higher_edu', 'incomplete_higher',
    'special_secondary', 'secondary', 'incomplete_secondary', 'no_edu', 'other_edu',
    'worker', 'gov_employee', 'official', 'military', 'student', 'not_working',
    'preschool', 'other_job', 'husband', 'wife', 'father', 'mother', 'son', 'daughter',
    'other_relative', 'group', 'alone', 'alcohol', 'narcotic', 'toxic', 'domestic'
]

# --- Обработка второго датасета (df2: совершившие насилие) ---
df2 = df2.iloc[:-1, :38]
df2.columns = [
    'region', 'total_perpetrators', 'physical', 'psychological', 'sexual', 'other',
    'male', 'female', 'boy', 'girl', 'age_14_18', 'age_18_35', 'age_35_50', 'age_50+',
    'higher_edu', 'incomplete_higher', 'special_secondary', 'secondary',
    'incomplete_secondary', 'illiterate', 'other_edu', 'worker', 'gov_employee',
    'military', 'student', 'not_working', 'other_job', 'husband', 'wife', 'father',
    'mother', 'son', 'daughter', 'other_relative', 'group', 'alone', 'alcohol', 'domestic'
]

# --- Обработка третьего датасета (df3: временные ряды) ---
df3 = df3.iloc[1:10, 2:]  # Только регионы и годы, начиная с третьего столбца
years = range(2009, 2024)
new_columns = ['region'] + [f'{y}_{g}' for y in years for g in ['total', 'female', 'male']]
if len(df3.columns) == len(new_columns):
    df3.columns = new_columns
else:
    raise ValueError(f"Количество столбцов не совпадает: {len(df3.columns)} != {len(new_columns)}")

# Преобразование данных в длинный формат
df3 = df3.melt(id_vars='region', var_name='year_gender', value_name='count')

# Разделение столбца year_gender на year и gender
df3[['year', 'gender']] = df3['year_gender'].str.split('_', expand=True)
df3['year'] = pd.to_numeric(df3['year'], errors='coerce')  # Преобразование года в числовой формат
df3['count'] = pd.to_numeric(df3['count'].replace('-', np.nan), errors='coerce')  # Замена '-' на NaN
df3 = df3.dropna(subset=['year', 'count'])  # Удаление строк с некорректными данными

# Поворот данных в широкий формат
df3 = df3.pivot_table(
    index=['region', 'year'],
    columns='gender',
    values='count',
    aggfunc='sum'
).reset_index()

# --- Объединение данных ---
# Для пострадавших (victims)
victim_data = df1.melt(
    id_vars=['court'],
    value_vars=[c for c in df1.columns if c not in ['court']],
    var_name='category',
    value_name='count'
)

# Для совершивших насилие (perpetrators)
perp_data = df2.melt(
    id_vars=['region'],
    value_vars=[c for c in df2.columns if c not in ['region']],
    var_name='category',
    value_name='count'
)

# Создание общего датасета
final_data = pd.concat([
    victim_data.assign(type='victim'),
    perp_data.assign(type='perpetrator')
])

# Добавление временных рядов
# Проверка наличия столбца 'year' в final_data
if 'year' not in final_data.columns:
    final_data['year'] = 2022  # Укажите год, связанный с данными

# Сопоставление court и region
court_to_region = {
    'Ленинский р/суд': 'Bishkek city',
    'Первомайский р/суд': 'Bishkek city',
    'Свердловский р/суд': 'Bishkek city',
    'Ташкумырский г/суд': 'Bishkek city',
    'Аламудунский р/суд': 'Chui oblast',
    'Сокулукский р/суд': 'Chui oblast',
    'Жайылский р/суд': 'Chui oblast',
    'Чуйский р/суд': 'Chui oblast',
    'Московский р/суд': 'Chui oblast',
    'Иссык-Атинский р/суд': 'Chui oblast',
    # Добавьте остальные соответствия
}
final_data['region'] = final_data['court'].map(court_to_region)

# Сопоставление названий регионов
region_mapping = {
    'г. Бишкек': 'Bishkek city',
    'Баткенская область': 'Batken oblast',
    'Джалал-Абадская область': 'Jalal-Abat oblast',
    'Иссык-Кульская область': 'Yssyk-Kul oblast',
    'Нарынская область': 'Naryn oblast',
    'Ошская область': 'Osh oblast',
    'Таласская область': 'Talas oblast',
    'Чуйская область': 'Chui oblast',
    'г. Ош': 'Osh city'
}
final_data['region'] = final_data['region'].map(region_mapping)

# Преобразование year в числовой формат
final_data['year'] = pd.to_numeric(final_data['year'], errors='coerce')

# Объединение с df3
final_data = final_data.merge(df3, left_on=['region', 'year'], right_on=['region', 'year'], how='left')

# Переименование столбцов
final_data.rename(columns={
    'total': 'total_combined',
    'female': 'female_combined',
    'male': 'male_combined'
}, inplace=True)

# Очистка данных
final_data = final_data.drop_duplicates()  # Удаление дубликатов
final_data = final_data.reset_index(drop=True)  # Сброс индекса

# Сохранение результата
final_data.to_csv('data/combined_domestic_violence_data.csv', index=False)
print("Объединение завершено. Данные сохранены в data/combined_domestic_violence_data.csv")

Объединение завершено. Данные сохранены в data/combined_domestic_violence_data.csv


C:\Users\User\AppData\Local\Temp\ipykernel_10476\2512119231.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df3['count'] = pd.to_numeric(df3['count'].replace('-', np.nan), errors='coerce')  # Замена '-' на NaN
